# PHQ615 : Calculs 

## Calcul de la connexion affine et du tenseur de courbure à partir de la métrique

## 2-sphère

Calculs de la connexion et du tenseur de courbure pour la 2-sphère en coordonnées sphériques $\theta$ et $\phi$.
On inscrit ensuite le tenseur métrique $g_{ij}$ et son inverse $g^{ij}$.

In [1]:
from sympy import *
from sympy import Array

init_printing()

a, θ, ϕ = symbols('a θ ϕ')
gd = Array([[a**2, 0],[0, a**2*sin(θ)**2]])
coords = Array([θ, ϕ])
gu = Array([[1/a**2, 0],[0, 1/(a**2*sin(θ)**2)]])
gd

⎡ 2            ⎤
⎢a       0     ⎥
⎢              ⎥
⎢     2    2   ⎥
⎣0   a ⋅sin (θ)⎦

In [2]:
gu

⎡1             ⎤
⎢──      0     ⎥
⎢ 2            ⎥
⎢a             ⎥
⎢              ⎥
⎢        1     ⎥
⎢0   ──────────⎥
⎢     2    2   ⎥
⎣    a ⋅sin (θ)⎦

On vérifie ici que la contraction $g_{ik}g^{kj}$ est bel et bien le delta de Kronecker $\delta^j_i$.

In [3]:
tensorcontraction(tensorproduct(gd, gu), (1,2))

⎡1  0⎤
⎢    ⎥
⎣0  1⎦

In [4]:
dg = derive_by_array(gd, coords)
dg

⎡⎡0          0         ⎤        ⎤
⎢⎢                     ⎥  ⎡0  0⎤⎥
⎢⎢      2              ⎥  ⎢    ⎥⎥
⎣⎣0  2⋅a ⋅sin(θ)⋅cos(θ)⎦  ⎣0  0⎦⎦

On calcul ici la connexion affine 

$$
\Gamma_{ijk} = \frac12\left(\partial_j g_{ik} + \partial_i g_{jk} - \partial_k g_{ij} \right)
$$

On commence par le dernier terme, à l'aide de la fonction `derive_by_array()`, ce qui, appliqué à la fonction $g_{ij}$ des coordonnées, donne son *gradient* $\partial_k g_{ij}$. L'ordre des coordonnée dans le tenseur d'ordre 3 ainsi obtenu est $(kij)$.

Ensuite, on permute les indices de l'expression obtenue de deux manière différentes (permutations (2,0,1) et (2,1,0)). Cela est fait via la fonction `permutedims()`. La permutation (2,0,1) signifie que l'ancien indice 0 ($=k$) s'en va en position 2, l'ancien indice 1 ($=i$) s'en va en position 0 et l'ancien indice 2 ($=j$) s'en va en position 1 : donc $(kij)$ devient $(ijk)$ et on obtient de cette manière $\partial_i g_{jk}$. La permutation (2,1,0) produit, quant à elle, $\partial_j g_{ik}$. On ajoute ces termes au premier et on divise par 2 pour obtenir $\Gamma_{ijk}$.

In [5]:
# connexion affine 'covariante'
dg = derive_by_array(gd, coords)
gammad = (permutedims(dg,(2,0,1)) + permutedims(dg,(2,1,0))-dg)/2
simplify(gammad)

⎡                    ⎡              2         ⎤⎤
⎢⎡0        0      ⎤  ⎢             a ⋅sin(2⋅θ)⎥⎥
⎢⎢                ⎥  ⎢     0       ───────────⎥⎥
⎢⎢     2          ⎥  ⎢                  2     ⎥⎥
⎢⎢   -a ⋅sin(2⋅θ) ⎥  ⎢                        ⎥⎥
⎢⎢0  ─────────────⎥  ⎢ 2                      ⎥⎥
⎢⎣         2      ⎦  ⎢a ⋅sin(2⋅θ)             ⎥⎥
⎢                    ⎢───────────       0     ⎥⎥
⎣                    ⎣     2                  ⎦⎦

Ensuite, on élève le premier indice de la connexion affine:

$$ \Gamma^i_{jk} = g^{il}\Gamma_{ljk} $$

In [6]:
# connexion affine
Γ = tensorcontraction(tensorproduct(gu, gammad), (1,2))
simplify(Γ)

⎡                 ⎡          1   ⎤⎤
⎢⎡0      0     ⎤  ⎢  0     ──────⎥⎥
⎢⎢             ⎥  ⎢        tan(θ)⎥⎥
⎢⎢   -sin(2⋅θ) ⎥  ⎢              ⎥⎥
⎢⎢0  ──────────⎥  ⎢  1           ⎥⎥
⎢⎣       2     ⎦  ⎢──────    0   ⎥⎥
⎣                 ⎣tan(θ)        ⎦⎦

## Calcul du tenseur de courbure

Le tenseur de Riemann est

$$
R^l{}_{kji} = 
    \partial_j\Gamma^l_{ki} - \partial_i\Gamma^l_{kj}  + \Gamma^m_{ki}\Gamma^l_{mj} - \Gamma^m_{kj}\Gamma^l_{mi}
$$

On calcul le gradient et la contraction avec l'ordre des indices $(lkji)$ en tête, pour ensuite appliquer des permutations et combiner les différents termes. L'effet des différentes permutations sur l'ordre des indices est le suivant:

- (1,2,0,3) : $(lkji) \to (jlki) $
- (1,2,3,0) : $(lkji) \to (ilkj) $
- (2,0,3,1) : $(lkji) \to (kilj) $
- (2,0,1,3) : $(lkji) \to (kjli) $


In [7]:
# tenseur de Riemann
R1 = derive_by_array(Γ, coords)
C = tensorcontraction(tensorproduct(Γ, Γ), (0,4))
R = permutedims(R1,(1,2,0,3)) - permutedims(R1,(1,2,3,0)) + permutedims(C,(2,0,3,1)) - permutedims(C,(2,0,1,3))
R = simplify(R)
R

⎡         ⎡             2   ⎤⎤
⎢⎡0  0⎤   ⎢   0      sin (θ)⎥⎥
⎢⎢    ⎥   ⎢                 ⎥⎥
⎢⎣0  0⎦   ⎢    2            ⎥⎥
⎢         ⎣-sin (θ)     0   ⎦⎥
⎢                            ⎥
⎢⎡0  -1⎤        ⎡0  0⎤       ⎥
⎢⎢     ⎥        ⎢    ⎥       ⎥
⎣⎣1  0 ⎦        ⎣0  0⎦       ⎦

Ici on abaisse le premier indice pour obtenir $R_{ijkl}$

In [8]:
# tenseur de Riemann covariant
Rcov =  tensorcontraction(tensorproduct(gd, R), (1,2))
Rcov

⎡                           ⎡              2    2   ⎤⎤
⎢         ⎡0  0⎤            ⎢     0       a ⋅sin (θ)⎥⎥
⎢         ⎢    ⎥            ⎢                       ⎥⎥
⎢         ⎣0  0⎦            ⎢  2    2               ⎥⎥
⎢                           ⎣-a ⋅sin (θ)      0     ⎦⎥
⎢                                                    ⎥
⎢⎡              2    2   ⎤                           ⎥
⎢⎢    0       -a ⋅sin (θ)⎥           ⎡0  0⎤          ⎥
⎢⎢                       ⎥           ⎢    ⎥          ⎥
⎢⎢ 2    2                ⎥           ⎣0  0⎦          ⎥
⎣⎣a ⋅sin (θ)       0     ⎦                           ⎦

In [9]:
# vérification des propriétés de symétrie de Rcov
A = Rcov + permutedims(Rcov,(0,2,3,1)) + permutedims(Rcov,(0,3,1,2))
simplify(A)

⎡⎡0  0⎤  ⎡0  0⎤⎤
⎢⎢    ⎥  ⎢    ⎥⎥
⎢⎣0  0⎦  ⎣0  0⎦⎥
⎢              ⎥
⎢⎡0  0⎤  ⎡0  0⎤⎥
⎢⎢    ⎥  ⎢    ⎥⎥
⎣⎣0  0⎦  ⎣0  0⎦⎦

In [10]:
# vérification de la nullité de la contraction des deux premiers indices
tensorcontraction(R, (0,1))

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

In [11]:
# tenseur de Ricci
Ricci = tensorcontraction(R, (0,2))
Ricci

⎡1     0   ⎤
⎢          ⎥
⎢      2   ⎥
⎣0  sin (θ)⎦

In [12]:
# Courbure scalaire
Rm = tensorcontraction(tensorproduct(gu,Ricci), (1,2))
Rscal = tensorcontraction(Rm, (0,1))
simplify(Rscal)

In [15]:
# tenseur d'Einstein
E = Ricci - gd*Rscal/2
E = simplify(E)
E

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

### Calcul de la dérivée covariante d'un tenseur A

In [24]:
def dcov(A, coords, pos):
    rank = len(pos)
    D1 = derive_by_array(A, coords)
    for i in range(rank):
        perm = list(range(rank+1))
        perm[i+1] = 0
        perm[0] = 1
        for j in range(1, i+1):
            perm[j] += 1
        if pos[i] is 0:
            print('indice ', i, ' est covariant, perm = ', tuple(perm))
            B = tensorcontraction(tensorproduct(Γ, A), (0, i+3))
            D1 -= permutedims(B, perm)
        else:
            print('indice ', i, ' est contravariant, perm = ', tuple(perm))
            B = tensorcontraction(tensorproduct(Γ, A), (1, i+3))
            D1 += permutedims(B, perm)
    return D1


In [27]:
simplify(dcov(gd, coords, (0,0)))

indice  0  est covariant, perm =  (1, 0, 2)
indice  1  est covariant, perm =  (1, 2, 0)


⎡⎡0  0⎤  ⎡0  0⎤⎤
⎢⎢    ⎥  ⎢    ⎥⎥
⎣⎣0  0⎦  ⎣0  0⎦⎦

In [25]:
# identité de Bianchi

B = dcov(Rcov, coords, (0,0,0,0))
permutedims(B, (1,2,3,4,0))+permutedims(B, (1,2,0,3,4))+permutedims(B, (1,2,4,0,3))

indice  0  est covariant, perm =  (1, 0, 2, 3, 4)
indice  1  est covariant, perm =  (1, 2, 0, 3, 4)
indice  2  est covariant, perm =  (1, 2, 3, 0, 4)
indice  3  est covariant, perm =  (1, 2, 3, 4, 0)


⎡⎡⎡0  0⎤  ⎡0  0⎤⎤  ⎡⎡0  0⎤  ⎡0  0⎤⎤⎤
⎢⎢⎢    ⎥  ⎢    ⎥⎥  ⎢⎢    ⎥  ⎢    ⎥⎥⎥
⎢⎢⎣0  0⎦  ⎣0  0⎦⎥  ⎢⎣0  0⎦  ⎣0  0⎦⎥⎥
⎢⎢              ⎥  ⎢              ⎥⎥
⎢⎢⎡0  0⎤  ⎡0  0⎤⎥  ⎢⎡0  0⎤  ⎡0  0⎤⎥⎥
⎢⎢⎢    ⎥  ⎢    ⎥⎥  ⎢⎢    ⎥  ⎢    ⎥⎥⎥
⎣⎣⎣0  0⎦  ⎣0  0⎦⎦  ⎣⎣0  0⎦  ⎣0  0⎦⎦⎦